In [12]:
import hail as hl

from hail.plot import show
from os import path
import os
import pandas as pd
import scipy.linalg
import numpy as np

hl.plot.output_notebook()

Loading BokehJS ...

In [13]:
#Useful command to specify nuimber of partitions:
def read_with_partitions(path, n_parts):
     mt = hl.read_matrix_table(path)
     return hl.read_matrix_table(path, _intervals=mt._calculate_new_partitions(n_parts))

In [14]:
# Set the percent of individuals in the UK Biobank to run the analysis on
pct = 1
print(str(pct)+"pct")
# if Refilter = False, use variables that are in storage (same SNPs, same individuals, no re-computations)
Refilter = False
# If ModifyGRM, use a local genetic_relatedness_matrix function and downweight by LD score
ModifyGRM = False

1pct


In [15]:
#Specify file names and locations:
genomt_file = "gs://ukb31063/ukb31063.genotype.mt"

withdrawn_file = "gs://ukb31063/ukb31063.withdrawn_samples.ht"
gwassamples_file ="gs://ukb31063/ukb31063.neale_gwas_samples.both_sexes.ht"
sampleqc_file = "gs://ukb31063/ukb31063.sample_qc.tsv.bgz"
snpqc_file = "gs://ukb-gt/ukb_snp_qc_forHail_chr1to22.txt"

gwasvariants_file = "gs://ukb31063/ukb31063.neale_gwas_variants.ht"

phenos_file = "gs://ukb31063/ukb31063.PHESANT_January_2019.both_sexes.tsv.bgz"
assessment_file = "gs://ukb-gt/ukb_acentre.csv"
covariates_file = "gs://ukb31063/ukb31063.neale_gwas_covariates.both_sexes.tsv.bgz"

pcvar_file = "gs://nbaya/sex_linreg/ukb31063.*_phenotypes.both_sexes.reg1.tsv.bgz"

# Saved locally on gs://ukb-gt
LDscore_file = "gs://ukb-gt/LDscores/"

In [ ]:
if ModifyGRM:
    # This function computes the GRM of a matrix while downweighting by the LD score of each SNP
    # Before calling this function, annotate ukbb as
    #   ukbb = ukbb.annotate_rows(LD_score = ld[ukbb.row_key].ld_score)
    def genetic_relatedness_matrix(ukbb) -> BlockMatrix:
        ukbb = ukbb.annotate_entries(n_alt = ukbb.GT.n_alt_alleles())
        ukbb = ukbb.unfilter_entries()
        ukbb = ukbb.annotate_rows(AC = agg.sum(ukbb.n_alt),n_called = agg.count_where(hl.is_defined(ukbb.n_alt)))
        ukbb = ukbb.filter_rows((ukbb.AC > 0) & (ukbb.AC < 2 * ukbb.n_called))
        ukbb = ukbb.annotate_rows(mean_n_alt = ukbb.AC / ukbb.n_called)
        ukbb = ukbb.annotate_rows(hwe_scaled_std_dev = hl.sqrt(ukbb.mean_n_alt * (2 - ukbb.mean_n_alt)))
        ukbb = ukbb.annotate_rows(normalizer=hl.sqrt(ukbb.mean_n_alt * (2 - ukbb.mean_n_alt) * ukbb.LD_score))
        normalized_n_alt = hl.or_else((ukbb.n_alt - ukbb.mean_n_alt) / ukbb.normalizer,0.0)
        bm = BlockMatrix.from_entry_expr(normalized_n_alt)
        grm = (bm.T @ bm) / (bm.n_rows / 2.0)
        return grm

In [16]:
# Define phenotype and assessment centre classification:
withdrawn = hl.read_table(withdrawn_file)
gwassamples = hl.read_table(gwassamples_file)
ld = hl.read_table(LDscore_file)

In [6]:
# Import table of SNPs used in PCA conducted by the UKB:
if Refilter:
    snpqc = hl.import_table(snpqc_file, delimiter=" ", impute=True, key='rs_id')
    snpqc = snpqc.filter(snpqc.in_PCA == 1, keep=True )  # only keep variants that have in_PCA == 1 (already passed LD pruning)
    snpqc = snpqc.annotate(locus=hl.locus(hl.str(snpqc.chromosome), snpqc.position, reference_genome='GRCh37'))
    snpqc = snpqc.key_by(snpqc.locus)

In [7]:
# Read ukbb genotype MatrixTable and filter
if Refilter:
    ukbb = hl.read_matrix_table(genomt_file)
    ukbb = ukbb.filter_cols(hl.is_defined(gwassamples[ukbb.col_key]), keep=True) #Keep only Neale lab GWAS samples
    ukbb = ukbb.filter_cols(hl.is_defined(withdrawn[ukbb.col_key]), keep=False) #withdrawn samples were already removed actually
    ukbb = ukbb.sample_cols(pct/100.) # (Lillian) filter number of individuals
    ukbb = hl.variant_qc(ukbb)
    ukbb = ukbb.filter_rows( (hl.min(ukbb.variant_qc.AF[0], ukbb.variant_qc.AF[1]) > 0.01) & ( ukbb.variant_qc.call_rate > 0.95), keep=True) # MAF > 0.01 and SNP call rate at least 95%
    ukbb = hl.sample_qc(ukbb)
    ukbb = ukbb.filter_cols( ukbb.sample_qc.call_rate > 0.98, keep=True) # sample missingness must be < 0.02
    ukbb = hl.variant_qc(ukbb)
    ukbb = ukbb.filter_rows( ukbb.variant_qc.call_rate > 0.98 , keep=True) # SNP missingness must be < 0.02 and hwe pvalue must be < 10^-6
    ukbb = ukbb.filter_rows( ukbb.variant_qc.p_value_hwe < 0.0000000001 , keep=False) # SNP missingness must be < 0.02 and hwe pvalue must be < 10^-6
    ukbb.count()

In [8]:
if Refilter:
    # Save Metadata (write any notes on processing steps, snps, individuals, etc)
    metadata = pd.DataFrame({'nindv': [ukbb.count()[0]], 'nsnps': [ukbb.count()[1]], 'processing': ['SNPs from Neale Lab GWAS study']})
    metadata.to_csv("gs://ukb-gt/"+str(pct)+"pct/ukbQC_"+str(ukbb.count()[0])+"_x_"+str(ukbb.count()[1])+".txt", index=False)
    
    # Write intermediate UKB matrix table to file:
    ukbQC_file = "gs://ukb-gt/"+str(pct)+"pct/ukbQC_"+str(ukbb.count()[0])+"_x_"+str(ukbb.count()[1])+".mt"
    ukbb.write(ukbQC_file, overwrite=True)
    ukbb.count()

In [9]:
# Read in UKB and with specified number of partitions:
if Refilter:
    ukbb = read_with_partitions(ukbQC_file, n_parts=500)
    ukbb.count() # counts are for quick checks that everything is correct
    
    # Filter SNPs to only those found in UKBB PCA
    ukbb = ukbb.filter_rows(hl.is_defined(snpqc[ukbb.locus]))
    
    # Write final UKB matrix table to file:
    ukbPCA_file = "gs://ukb-gt/"+str(pct)+"pct/ukbPCA_"+str(ukbb.count()[0])+"_x_"+str(ukbb.count()[1])+".mt"
    ukbb.write(ukbPCA_file, overwrite=True)

In [10]:
# If using old saved data, read it in
if not Refilter:
    # set number of SNPs and individuals by hand
    if pct==10: 
        nsnps = 575904
        nindv = 35946
    elif pct==5:
        nsnps = 145061
        nindv = 17799
    elif pct==1:
        nsnps = 145089
        nindv = 3621
    ukbPCA_file = "gs://ukb-gt/"+str(pct)+"pct/ukbPCA_"+str(nsnps)+"_x_"+str(nindv)+".mt"
    ukbb = hl.import_table(ukbPCA_file)

NameError: name 'nindvtmp' is not defined

In [ ]:
if ModifyGRM:
    # annotate LD scores into UKBB (WARNING: this might be the incorrect LD scores)
    ukbb = ukbb.annotate_rows(LD_score = ld[ukbb.row_key].ld_score)

In [28]:
# compute and save the genetic relatedness matrix
if ModifyGRM:
    grm = genetic_relatedness_matrix(ukbb)
else:
    grm = hl.genetic_relatedness_matrix(ukbb.GT)
grm_file = "gs://ukb-gt/"+str(pct)+"pct/grm_"+str(ukbb.count()[0])+"_x_"+str(ukbb.count()[1])+".mt"
grm.write(grm_file, overwrite=True)

2021-02-14 18:25:15 Hail: INFO: Coerced sorted dataset
2021-02-14 18:26:00 Hail: INFO: Wrote all 36 blocks of 145089 x 3621 matrix with block size 4096.
2021-02-14 18:26:02 Hail: INFO: Coerced sorted dataset
2021-02-14 18:26:09 Hail: INFO: Coerced sorted dataset
2021-02-14 18:28:26 Hail: INFO: wrote matrix with 3621 rows and 3621 columns as 1 block of size 4096 to gs://ukb-gt/1pct/grm_145089_x_3621_meta0.mt


In [31]:
# Convert the grm to a numpy array and save it
nsnps = ukbb.count()[0]
nindv = ukbb.count()[1]

grm_np = grm.to_numpy()
np.save("/tmp/grm_"+str(nsnps)+"_x_"+str(nindv)+".npy", grm_np)
hl.hadoop_copy("file:///tmp/grm_"+str(nsnps)+"_x_"+str(nindv)+".npy", "gs://ukb-gt/"+str(pct)+"pct/grm_"+str(nsnps)+"_x_"+str(nindv)+".npy")

2021-02-14 18:38:01 Hail: INFO: Coerced sorted dataset
2021-02-14 18:38:20 Hail: INFO: Coerced sorted dataset


In [32]:
# Eigenvalue decomposition
eigenvals = np.linalg.eigvalsh(grm_np)
np.save("/tmp/eigenvals_"+str(nsnps)+"_x_"+str(nindv)+".npy", eigenvals)
hl.hadoop_copy("file:///tmp/eigenvals_"+str(nsnps)+"_x_"+str(nindv)+".npy", "gs://ukb-gt/"+str(pct)+"pct/eigenvals_"+str(nsnps)+"_x_"+str(nindv)+".npy")

2021-02-14 18:39:24 Hail: INFO: Coerced sorted dataset
2021-02-14 18:39:31 Hail: INFO: Coerced sorted dataset


In [1]:
# Plot eigenvalues overlayed with Marchenko-Pastur
import matplotlib.pyplot as plt
import math

# Compute Marchenko-Pastur
lmda = nindv / nsnps
lmdap = (1 + np.sqrt(lmda))**2
lmdam = (1 - np.sqrt(lmda))**2
x = np.arange(lmdam, lmdap, 0.001)
y = (1/(2*math.pi)) * np.sqrt((lmdap-x)*(x-lmdam)) / (lmda*x)

# Plot
plt.clf()
plt.hist(eigenvals[1:], bins=int(nindv/35), density = True)
plt.plot(x, y, '-b', label = 'Marchenko-Pastur Distrubution')
plt.title('Eigenvalues for '+str(nindv)+' Individuals ('+str(pct)+'%) and '+str(nsnps)+' SNPs')
plt.xlabel('Eigenvalues')
plt.ylabel('Density')
plt.xlim([0,2.75])
plt.grid(True)
plt.legend()
plt.savefig('/tmp/eigenval_dist_'+str(nindv)+'_x_'+str(nsnps)+'.pdf')
hl.hadoop_copy('file:///tmp/eigenval_dist_'+str(nindv)+'_x_'+str(nsnps)+'.pdf', 'gs://ukb-gt/'+str(pct)+'pct/eigenval_dist_'+str(nindv)+'_x_'+str(nsnps)+'.pdf')
plt.show()

NameError: name 'nindv' is not defined